In [2]:
import pandas as pd

df = pd.read_csv('C:/Users/LSNAu/Desktop/seoul_apratment_price.csv', encoding='cp949')
df.info()

C:\Users\LSNAu\AppData\Local\Temp\ipykernel_55776\290852887.py:3: DtypeWarning: Columns (7,8,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/LSNAu/Desktop/seoul_apratment_price.csv', encoding='cp949')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331744 entries, 0 to 1331743
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   접수연도              1331744 non-null  int64  
 1   자치구코드             1331744 non-null  int64  
 2   자치구명              1331743 non-null  object 
 3   법정동코드             1331744 non-null  int64  
 4   법정동명              1331744 non-null  object 
 5   지번구분              1331617 non-null  float64
 6   지번구분명             1331617 non-null  object 
 7   본번                1331617 non-null  object 
 8   부번                1331617 non-null  object 
 9   건물명               1331744 non-null  object 
 10  계약일               1331744 non-null  int64  
 11  물건금액(만원)          1331744 non-null  int64  
 12  건물면적(㎡)           1331744 non-null  float64
 13  토지면적(㎡)           366930 non-null   float64
 14  층                 1331744 non-null  int64  
 15  권리구분              55903 non-null    object 
 16  

In [6]:
df['법정동명'].nunique()

337

In [8]:
import googlemaps
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

# Google Maps 클라이언트 초기화
gmaps = googlemaps.Client(key='Enter_API')

# 고유한 법정동명 추출
unique_addresses = df['법정동명'].unique()

def geocode_address(address):
    geocode_result = gmaps.geocode(address)
    if geocode_result:
        latitude = geocode_result[0]['geometry']['location']['lat']
        longitude = geocode_result[0]['geometry']['location']['lng']
        return (address, latitude, longitude)
    else:
        return (address, None, None)

# ThreadPoolExecutor를 사용하여 요청 병렬 처리
results = []
with ThreadPoolExecutor(max_workers=14) as executor:
    futures = {executor.submit(geocode_address, address): address for address in unique_addresses}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Geocoding"):
        result = future.result()
        results.append(result)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results, columns=['법정동명', '위도', '경도'])

# 원본 df에 위도와 경도 매핑
df = pd.merge(df, results_df, on='법정동명', how='left')

# 결과 출력
df.info()

Geocoding: 100%|██████████| 337/337 [00:05<00:00, 61.32it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331744 entries, 0 to 1331743
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   접수연도              1331744 non-null  int64  
 1   자치구코드             1331744 non-null  int64  
 2   자치구명              1331743 non-null  object 
 3   법정동코드             1331744 non-null  int64  
 4   법정동명              1331744 non-null  object 
 5   지번구분              1331617 non-null  float64
 6   지번구분명             1331617 non-null  object 
 7   본번                1331617 non-null  object 
 8   부번                1331617 non-null  object 
 9   건물명               1331744 non-null  object 
 10  계약일               1331744 non-null  int64  
 11  물건금액(만원)          1331744 non-null  int64  
 12  건물면적(㎡)           1331744 non-null  float64
 13  토지면적(㎡)           366930 non-null   float64
 14  층                 1331744 non-null  int64  
 15  권리구분              55903 non-null    object 
 16  

In [10]:
df.to_csv('seoul_apratment_price_lalo.csv', encoding='utf-8')